In [1]:
import numpy as np

In [2]:
seed = 0
n, p = int(6e3), int(5e3)
X = np.asfortranarray(np.random.randn(n, p))
H = X.T @ X / n

In [3]:
import choosi as cs

In [4]:
beta = np.random.choice([-1,1], size=p, replace=True) * np.random.choice([3,4,5], size=p, replace=True)

In [5]:
y = X @ beta + np.random.randn(n)

In [6]:
v = -X.T @ y / n
scaling = np.ones(p)

In [7]:
x_soln = np.linalg.solve(H, -v)

In [8]:
signs = np.sign(x_soln) 

In [13]:
qnm = cs.optimizer.CQNMOptimizer(v, H, signs, scaling, lmda=1e-6)

In [14]:
x_opt = qnm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

In [159]:
%%timeit
qnm = cs.optimizer.CQNMOptimizer(v, H, signs, scaling, lmda=1e-6)
x_opt = qnm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

763 ms ± 40.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [160]:
nm = cs.optimizer.NMOptimizer(v, H, signs, scaling, lmda=1e-6)

In [161]:
x_opt = nm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

In [ ]:
%%timeit
nm = cs.optimizer.NMOptimizer(v, H, signs, scaling, lmda=1e-6)
x_opt = nm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

In [37]:
H_chol.dtype

dtype('float64')

In [139]:
%%time
H_chol = np.linalg.cholesky(H)
qnm_cpp = cs.choosi_core.optimization.QPBarrierCQN(
    quad=H, 
    linear=v, 
    signs=signs, 
    lmda=1e-6, 
    n_threads=8, 
    quad_chol=H_chol, 
    max_iters=100, 
    tol=1e-10, 
    armijo=False, 
    armijo_max_iters=100, 
    armijo_c=0.5, 
    armijo_tau=0.5,
)

CPU times: user 4.52 s, sys: 1.3 s, total: 5.82 s
Wall time: 720 ms


In [58]:
qnm = cs.optimizer.CQNMOptimizer(v, H, signs, scaling, lmda=1e-6)

In [59]:
%time x_opt = qnm.optimize(max_steps=100, tau=0.5, c=0.5, tol=1e-10)

CPU times: user 4.76 s, sys: 2.06 s, total: 6.82 s
Wall time: 745 ms


In [146]:
%time x_opt_cpp = qnm_cpp.solve()

CPU times: user 250 ms, sys: 28.8 ms, total: 279 ms
Wall time: 35.5 ms


In [162]:
%%time
H_chol = np.linalg.cholesky(H)
qnm_cpp = cs.choosi_core.optimization.QPBarrierCQN(
    quad=H, 
    linear=v, 
    signs=signs, 
    lmda=1e-6, 
    n_threads=8, 
    quad_chol=H_chol, 
    max_iters=100, 
    tol=1e-10, 
    armijo=False, 
    armijo_max_iters=100, 
    armijo_c=0.5, 
    armijo_tau=0.5,
)

CPU times: user 6 s, sys: 1.24 s, total: 7.24 s
Wall time: 961 ms


In [163]:
%%timeit
qnm_cpp.solve()

33.4 ms ± 461 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%prun -s cumtime
qnm = cs.optimizer.CQNMOptimizer(v, H, signs, scaling, lmda=1e-6)
x_opt = qnm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

         972 function calls (968 primitive calls) in 0.084 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.084    0.084 {built-in method builtins.exec}
        1    0.000    0.000    0.084    0.084 <string>:1(<module>)
        1    0.000    0.000    0.084    0.084 optimizer.py:55(optimize)
        1    0.000    0.000    0.084    0.084 optimizer.py:70(_optimize)
        4    0.000    0.000    0.069    0.017 optimizer.py:192(_get_hessinv)
        1    0.063    0.063    0.063    0.063 linalg.py:688(cholesky)
     12/8    0.000    0.000    0.010    0.001 _interface.py:205(matvec)
        8    0.000    0.000    0.010    0.001 _interface.py:592(_matvec)
        8    0.000    0.000    0.010    0.001 optimizer.py:196(trisolve_matvec)
       16    0.009    0.001    0.009    0.001 _basic.py:264(solve_triangular)
        5    0.007    0.001    0.007    0.001 optimizer.py:50(_get_grad)
        4    0.

In [15]:
x_opt

array([ 3.01285994,  3.99549768, -2.98756779, ..., -2.97888004,
        2.9748843 , -4.9867582 ])

In [16]:
x_soln

array([ 3.01298849,  3.99567741, -2.98763031, ..., -2.97914608,
        2.97502644, -4.98700454])

In [17]:
np.allclose(x_opt, x_soln)

False

In [18]:
qnm_obj = .5 * x_opt.T.dot(H.dot(x_opt)) - v.dot(x_opt)

In [19]:
%%prun -s cumtime
nm = cs.optimizer.NMOptimizer(v, H, signs, scaling, lmda=1e-6)
x_opt = nm.optimize(max_steps=100, tau=.5, c=.5, tol=1e-10)

         4170 function calls (4146 primitive calls) in 4.566 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.566    4.566 {built-in method builtins.exec}
        1    0.000    0.000    4.566    4.566 <string>:1(<module>)
        1    0.000    0.000    4.566    4.566 optimizer.py:55(optimize)
        1    0.001    0.001    4.566    4.566 optimizer.py:70(_optimize)
    72/48    0.000    0.000    4.416    0.092 _interface.py:205(matvec)
       48    0.000    0.000    4.415    0.092 _interface.py:592(_matvec)
       48    0.000    0.000    4.415    0.092 optimizer.py:35(solve_matvec)
       48    4.414    0.092    4.414    0.092 linalg.py:329(solve)
       24    0.002    0.000    2.254    0.094 optimizer.py:30(_get_hessinv)
       24    0.000    0.000    2.250    0.094 _interface.py:573(__init__)
       24    0.000    0.000    2.249    0.094 _interface.py:177(_init_dtype)
       24    0.000   

In [20]:
x_opt

array([ 3.01284608,  3.9954782 , -2.98756107, ..., -2.97885121,
        2.97486888, -4.98673145])

In [21]:
x_soln

array([ 3.01298849,  3.99567741, -2.98763031, ..., -2.97914608,
        2.97502644, -4.98700454])

In [22]:
np.allclose(x_opt, x_soln)

False

In [23]:
nm_obj = .5 * x_opt.T.dot(H.dot(x_opt)) - v.dot(x_opt)

In [24]:
np.allclose(qnm_obj, nm_obj)

True